# 다나와에서 자동차 정보 Crawling 하기

beautifulsoup로 하려 했으나 html 소스 파싱이 안되는 문제가 발생 -> 시간나면 확인해 볼것 -> lxml으로 파싱하면 됨 -> 해결
```python
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome('chromedriver.exe')
html = driver.page_source # html으로 파싱이 잘 안됨

soup = BeautifulSoup(html, 'html.parser') 
link_path = soup.select
modelList = soup.find_all('td', {'class':'title'})

print(soup)
for i in modelList:
print(i.a.string)
```

먼저 자동차 모델명을 가져오는 프로그램을 만들어 봄.
```python
from selenium import webdriver
import re

driver = webdriver.Chrome('chromedriver.exe')
model_name=[]
model_number=[]

# webdriver를 사용해서 chrome 브라우저에서 url을 연다
driver.get('http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2018-02-00')

model_list = driver.find_elements_by_class_name("title")

i=0
for m1 in model_list:
    # a tag가 있을때만 link에 하이퍼링크 주소 할당
    try: 
        link = m1.find_element_by_tag_name("a").get_attribute("href")
    except:
        continue
    # model_name list에 자동차 모델명 할당
    model_name.append (m1.text)
    # model_number list에 자동차 모델과 mapping되는 모델번호 할당 (spec 주소 찾는데 쓸 것)
    # re.compile 클래스는 정규표현식을 사용하여 text에서 정규표현식에 해당하는 값을 리턴함
    model_number.append (re.compile(r"Model=(\d+)").search(link).group(1))
    print(i,model_name[i], model_number[i])
    i+=1    
```  
결과예시 ->

1 그랜저 3566 

2 포터2 ...

이런식으로 잘 나옴

2016년1월부터~ 2018년 2월까지 돌리고 나면 중복이 발생할텐데 중복 제거해야함

dictionary type에 저장함으로써 해결가능해 보임


start_ym부터 end_ym까지 조회하면서 차량 모델명 및 모델번호를 딕셔너리 변수에 저장하는 프로그램

In [2]:
# import
from selenium import webdriver  
from bs4 import BeautifulSoup
import pandas as pd
import xml
import sys
import re

이상하게 웹드라이버 경로 지정하는 부분은 부분은 cell 마다 적어줘야 함. 왜지???
```python
# 크롬 웹드라이버 경로 지정
driver = webdriver.Chrome('chromedriver.exe')
```

In [3]:
# 크롬 웹드라이버 경로 지정
driver = webdriver.Chrome('chromedriver.exe')

model_name=[]
model_number=[]
model_dic=dict() 

# 조회 시작 년월
start_ym = pd.Period('2016-01','M')

# 조회 종료 년월 -> 전월
#end_ym = pandas.Period(pandas.datetime.today().strftime('%Y-%m'),'M') -1
# 2018년도 2월까지 조회
END_YM = pd.Period('2018-02','M')

url='http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=' # 이 뒤에 YYYY-MM-00 형식으로 조회년월 들어감

i=0
for strd_ym in pd.period_range(start_ym, END_YM, freq='M'):
    
    try:
        print('http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=' + strd_ym.strftime('%Y-%m-00'))
        driver.get('http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=' + strd_ym.strftime('%Y-%m-00'))
        #print('http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=' + strd_ym.strftime('%Y-%m-00'))
    except:
        print("조회 페이지 없음")
        continue
    
    model_list = driver.find_elements_by_class_name("title")

    for m1 in model_list:
        # a tag가 있을때만 link에 하이퍼링크 주소 할당
        try: 
            link = m1.find_element_by_tag_name("a").get_attribute("href")
            # model_number list에 자동차 모델과 mapping되는 모델번호 할당 (spec 주소 찾는데 쓸 것)
            # re.compile 클래스는 정규표현식을 사용하여 text에서 정규표현식에 해당하는 값을 리턴함
            model_number.append (re.compile(r"Model=(\d+)").search(link).group(1))
        except:
            continue
        # model_name list에 자동차 모델명 할당
        model_name.append (m1.text)
        # 중복제거를 위해 model_number와 model_name을 딕셔너리에 저장 (key - model_number, value - model_name)
        model_dic[model_number[i]] = model_name[i]
        i+=1    



http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-01-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-02-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-03-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-04-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-05-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-06-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-07-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-08-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-09-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-10-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-11-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2016-12-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-01-00
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-02-00
http://auto.danawa.com/auto/?Work=record&Tab=Mod

In [ ]:
print(model_dic)

여기까지 모델명을 받아오고, model number를 key로 하는 딕셔너리를 만들었음

다음으로는 model number를 이용하여 자동차 spec 페이지를 크롤링하는게 목표

그 전에, model에 lineup도 있다. 이건 어떻게 처리할지도 고민
* key를 modelNumber, value를 linup의 set으로 하는 dictionary를 하나 더 만들면 어떨까?
* 라인업별 스펙까지 필요한지는 모르겠으니 이건 2순위로 구현하자


``` python
#제원 버튼 클릭하기

lineupButton = driver.find_elements_by_class_name("buttonSelect")
lineupButton.click()
```


일단은 model에서 spec 테이블 긁어오는 것부터 진행

In [5]:
model_dic_backup= model_dic
#print(model_dic_backup)
#model_dic = model_dic_backup

#test data 만들기
model_dic_test = {'3298': '스포티지', '3273':'싼타페'}

아래 cell을 실행할때마다 test data와 실제 data가 바뀜

#test data와 원래 data swap
temp_dic = model_dic
model_dic = model_dic_test
model_dic_test = temp_dic

In [6]:
print(model_dic)

{'1901': '포터2', '3153': '쏘렌토', '3259': '아반떼', '3152': '카니발', '3239': '더 뉴 모닝', '3273': '싼타페', '3151': '쏘나타 (2017)', '1907': '봉고3 트럭', '3298': '스포티지', '3250': '투싼', '2129': '그랜저HG', '3255': '스파크', '2009': '스타렉스', '3260': 'K5', '3154': '티볼리', '3321': 'K3', '2588': 'K3', '3119': '제네시스(DH)', '3316': 'EQ900', '3039999': '버스/트럭 (현대)', '2189': '코란도 스포츠', '2183': '레이 (2017)', '3277': '임팔라', '3096': 'E-class', '2682': 'K7', '2126': '엑센트', '3300': '맥스크루즈', '3128': 'The S-class', '3270': 'A6', '2140': '올란도', '3105': '뉴 코란도 C', '2351': '5 Series', '3242': '뉴 크루즈', '2103': '쏘나타(YF)', '2337': 'Tiguan', '3138': '그랜저HG HEV', '3315': 'GLC', '3080': 'QM3', '3054': '트랙스', '3314': 'K5 하이브리드', '3301': '3-series', '2180': '말리부', '3238': 'SM5', '3193': 'LF쏘나타 HEV', '3326': '아이오닉 hybrid', '3095': 'Golf', '3191': 'SM3', '2591': 'ES', '2049': '다마스', '3274': 'Explorer', '3055': '코란도 투리스모', '3210': 'SM7', '3146': 'Q50', '2052': '라보', '3140': 'C-class', '3281': 'GLE', '2214': '렉스턴 W', '3237': '더 뉴 프라이드', '3246': '

In [8]:
# 크롬 웹드라이버 경로 지정
driver = webdriver.Chrome('chromedriver.exe')

#http://auto.danawa.com/auto/modelPopup.php?Type=spec&Lineup=42742&Trims=
# url1과 url2 사이에 모델 number가 들어가야함
url1='http://auto.danawa.com/auto/?Work=model&Model=' 
url2='&Tab=spec'
table=pd.DataFrame()
tabletitle=[]
tablecol=[[]]
tablerow=[[]]
dtl_model=[[]]
fail_model=[[],[]]
model_kr_name=[[]]
price=[[]]

# 모델 수
cnt=0
#print(model_dic.keys())

for mnum in model_dic.keys():
    try:
        driver.get(url1+mnum+url2)
    except:
        print("조회 페이지 없음")
        continue
    
    html = driver.page_source 
    table_src = BeautifulSoup(html, 'lxml') 
    # 표 부분 가져오기
    
    if table_src.find('div', id='printTable') != None:
        if table_src.find('div', {'class':'model_data_table'}) == None:
            print("테이블 양식 확인 => ", url1+mnum+url2)
            fail_model[0].append(mnum)
            continue
        else:
            table_src = table_src.find('div', {'class':'model_data_table'})
    else:
        print("테이블 가져오기 실패 => ", url1+mnum+url2)
        fail_model[1].append(mnum)
        continue
        
    # 표에서 세부 모델명 가져오기
    j=0
    try:
        table_src.find('ul', {'class':'modelname'}).find_all("li")
    except:
        print("테이블 가져오기 실패2 => ", url1+mnum+url2)
        fail_model[1].append(mnum)
        continue
    
    print(url1+mnum+url2)
    for dm1 in table_src.find('ul', {'class':'modelname'}).find_all("li"):
        j+=1
        if dm1.text == '-':
            continue
        else:
            dtl_model[cnt].append(dm1.text)
            model_kr_name[cnt].append(model_dic[mnum])
    
    max_i = len(dtl_model[cnt])
    
    # 표에서 가격 가져오기
    i=0
    for pri in table_src.find('ul', {'class':'price'}).find_all("li"):
        if i < max_i:
            price[cnt].append(pri.text)
        else:
            break
        i+=1
        
    
    # 표 그룹 (엔진, 전기, 구동, 연비, 연비(전기), 제원, 적재함, 성능) 가져오기
    group = table_src.find_all('div', {"class":"row group"})
    #print(group)
    
    for grp1 in group:
        tablecol[cnt].append(grp1.span.text)
        i=0
        for grp2 in grp1.find_all("li"):
            if i%j == 0 and len(tabletitle) < 53 : 
                tabletitle.append(grp2.text.replace("\t","").replace("\n",""))
            elif i%j != 0 and i%j < max_i:
                tablerow[cnt].append(grp2.text.replace("\t","").replace("\n",""))
            i+=1
    cnt+=1

    if len(tabletitle) != 53:
        print("테이블 행 개수 예외발생 => url1+mnum+url2")
        fail_model[1].append(mnum)
        break

    if cnt < len(model_dic.keys()):
        dtl_model.append([])
        tablerow.append([])
        tablecol.append([])
        model_kr_name.append([])
        price.append([])
    


http://auto.danawa.com/auto/?Work=model&Model=1901&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=3153&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3259&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3152&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=3239&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3273&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=3151&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=1907&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3298&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3250&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=2129&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3255&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=2009&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3260&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3154&Tab=spec
http://auto.danawa.com/auto/?Work=model&Mode

http://auto.danawa.com/auto/?Work=model&Model=3252&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=2246&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3251&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3319&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3297&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3235&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=2360&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=2545&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=3168&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=2468&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=2354&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=2688&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=2491&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3222&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3280&Tab=spec
http://auto.danawa.com/auto/?W

http://auto.danawa.com/auto/?Work=model&Model=3353&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3309&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3336&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3330&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=2619&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3383&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3375&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3389&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3247&Tab=spec
테이블 양식 확인 =>  http://auto.danawa.com/auto/?Work=model&Model=3144&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3391&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3379&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3269&Tab=spec
테이블 가져오기 실패 =>  http://auto.danawa.com/auto/?Work=model&Model=4139999&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3395&Tab=spec
http://auto.danawa.com/auto/?Work=model&Model=3403&Tab=spec
http://

아래와 같이 정렬하고 싶음.. -> 어떡하지??

모델|세부모델|가격|엔진형식|연료|배기량 (cc)|최고출력 (ps/rpm)|...
-|-|-|-|-|-|-|-|
스포티지 | 트렌디 (A/T) |23,200,000| U2 1.7 E-VGT|디젤|1,685|141/4,000|...
| 노블레스(A/T)|25,300,000|U2 1.7 E-VGT|디젤|1,685|141/4,0000|...





In [9]:
#'모델' # (1,1)
#print(len(model_dic.keys()))
#print(model_kr_name)
#print(dtl_model) # (1,2), (2,2) , (3,2)
#print(price) # (1,3), (2,3) ,(3,3)
#print(tabletitle) # (1,4),(1,5),...,(1,n)
#print(tablecol) # 쓸모없음
#print(tablerow) # (2,4),(3,4),(2,5),(3,5)...
#print(tabletitle)
print(len(model_dic.keys()))
print(len(dtl_model))
print(len(tabletitle))
print(tablerow)

361
251
53
[['2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '2.5 CRDi', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '디젤', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '2,497', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600', '133/3,600'

```python
from openpyxl import Workbook


wb = Workbook()
ws1 = wb.active # 현재 열려 있는 Sheet
ws1.title = "first_Sheet"  #시트 ws1의 이름을 지정
ws1["A1"] = "Hello World"  #셀 A1에 입력 
print(ws1["A1"].value)  #셀 A1의 내용 출력
wb.save("C:/excel/test.xlsx") #저장
```

In [11]:
# 실패한 모델 조회
#fail_model[0] => 양식다름
print(len(fail_model[0]),len(fail_model[1]),len(model_dic.keys()))
print(len(dtl_model)-1)
#print(tablerow[1])

101 10 361
250


In [12]:
# 엑셀에 쓰기
from openpyxl import Workbook


wb = Workbook()
ws1 = wb.active
ws1.title = "car_spec"


# 첫번째 row 처리
#print(ws1.cell(row=1,column=2))  #B1
ws1.cell(row=1,column=1).value = '모델명'
ws1.cell(row=1,column=2).value = '세부모델명'
ws1.cell(row=1,column=3).value = '가격'

for i in range(0,len(tabletitle)) :
    ws1.cell(row=1,column=4+i).value = tabletitle[i]

pt1 = 2

# 첫번째, 두번째, 세번째 컬럼처리
for i in range(0,len(dtl_model)):   # 0,1
    for j in range(1,len(dtl_model[i])): # 1,2
        ws1.cell(row=pt1, column=1).value = model_kr_name[i][j] #모델명
        ws1.cell(row=pt1, column=2).value = dtl_model[i][j]     #세부모델
        ws1.cell(row=pt1, column=3).value = price[i][j]         # 가격
        pt1+=1

# 네번째 컬럼부터 처리
pt2=2
for i in range(0,len(dtl_model)):
    ix=0
    for j in range(0,len(tabletitle)):
        for k in range(0,len(dtl_model[i])-1):
            print(i,ix,pt2+k,4+j,tablerow[i][ix])
            ws1.cell(row=pt2+k, column = 4+j).value = tablerow[i][ix]
            ix+=1
            ix=ix%(len(tabletitle)*(len(dtl_model[i])-1))
    pt2+=(len(dtl_model[i])-1)





0 0 2 4 2.5 CRDi
0 1 3 4 2.5 CRDi
0 2 4 4 2.5 CRDi
0 3 5 4 2.5 CRDi
0 4 6 4 2.5 CRDi
0 5 7 4 2.5 CRDi
0 6 8 4 2.5 CRDi
0 7 9 4 2.5 CRDi
0 8 10 4 2.5 CRDi
0 9 11 4 2.5 CRDi
0 10 12 4 2.5 CRDi
0 11 13 4 2.5 CRDi
0 12 14 4 2.5 CRDi
0 13 15 4 2.5 CRDi
0 14 16 4 2.5 CRDi
0 15 17 4 2.5 CRDi
0 16 18 4 2.5 CRDi
0 17 19 4 2.5 CRDi
0 18 20 4 2.5 CRDi
0 19 21 4 2.5 CRDi
0 20 22 4 2.5 CRDi
0 21 23 4 2.5 CRDi
0 22 24 4 2.5 CRDi
0 23 25 4 2.5 CRDi
0 24 26 4 2.5 CRDi
0 25 27 4 2.5 CRDi
0 26 28 4 2.5 CRDi
0 27 2 5 디젤
0 28 3 5 디젤
0 29 4 5 디젤
0 30 5 5 디젤
0 31 6 5 디젤
0 32 7 5 디젤
0 33 8 5 디젤
0 34 9 5 디젤
0 35 10 5 디젤
0 36 11 5 디젤
0 37 12 5 디젤
0 38 13 5 디젤
0 39 14 5 디젤
0 40 15 5 디젤
0 41 16 5 디젤
0 42 17 5 디젤
0 43 18 5 디젤
0 44 19 5 디젤
0 45 20 5 디젤
0 46 21 5 디젤
0 47 22 5 디젤
0 48 23 5 디젤
0 49 24 5 디젤
0 50 25 5 디젤
0 51 26 5 디젤
0 52 27 5 디젤
0 53 28 5 디젤
0 54 2 6 2,497
0 55 3 6 2,497
0 56 4 6 2,497
0 57 5 6 2,497
0 58 6 6 2,497
0 59 7 6 2,497
0 60 8 6 2,497
0 61 9 6 2,497
0 62 10 6 2,497
0 63 11 6 2,497
0 64 12 6 

0 918 2 38 -
0 919 3 38 -
0 920 4 38 -
0 921 5 38 -
0 922 6 38 -
0 923 7 38 -
0 924 8 38 -
0 925 9 38 -
0 926 10 38 -
0 927 11 38 -
0 928 12 38 -
0 929 13 38 -
0 930 14 38 -
0 931 15 38 -
0 932 16 38 -
0 933 17 38 -
0 934 18 38 -
0 935 19 38 -
0 936 20 38 -
0 937 21 38 -
0 938 22 38 -
0 939 23 38 -
0 940 24 38 -
0 941 25 38 -
0 942 26 38 -
0 943 27 38 -
0 944 28 38 -
0 945 2 39 -
0 946 3 39 -
0 947 4 39 -
0 948 5 39 -
0 949 6 39 -
0 950 7 39 -
0 951 8 39 -
0 952 9 39 -
0 953 10 39 -
0 954 11 39 -
0 955 12 39 -
0 956 13 39 -
0 957 14 39 -
0 958 15 39 -
0 959 16 39 -
0 960 17 39 -
0 961 18 39 -
0 962 19 39 -
0 963 20 39 -
0 964 21 39 -
0 965 22 39 -
0 966 23 39 -
0 967 24 39 -
0 968 25 39 -
0 969 26 39 -
0 970 27 39 -
0 971 28 39 -
0 972 2 40 5,100
0 973 3 40 5,100
0 974 4 40 5,100
0 975 5 40 5,100
0 976 6 40 5,100
0 977 7 40 5,100
0 978 8 40 5,155
0 979 9 40 5,155
0 980 10 40 4,775
0 981 11 40 4,775
0 982 12 40 5,135
0 983 13 40 5,135
0 984 14 40 5,135
0 985 15 40 5,135
0 986 16 40 5,13

1 287 34 51 -
1 288 29 52 -
1 289 30 52 -
1 290 31 52 -
1 291 32 52 -
1 292 33 52 -
1 293 34 52 -
1 294 29 53 -
1 295 30 53 -
1 296 31 53 -
1 297 32 53 -
1 298 33 53 -
1 299 34 53 -
1 300 29 54 -
1 301 30 54 -
1 302 31 54 -
1 303 32 54 -
1 304 33 54 -
1 305 34 54 -
1 306 29 55 -
1 307 30 55 -
1 308 31 55 -
1 309 32 55 -
1 310 33 55 -
1 311 34 55 -
1 312 29 56 -
1 313 30 56 -
1 314 31 56 -
1 315 32 56 -
1 316 33 56 -
1 317 34 56 -
2 0 35 4 R 2.2 E-VGT
2 1 36 4 R 2.2 E-VGT
2 2 37 4 R 2.2 E-VGT
2 3 35 5 디젤
2 4 36 5 디젤
2 5 37 5 디젤
2 6 35 6 2,199
2 7 36 6 2,199
2 8 37 6 2,199
2 9 35 7 202/3,800
2 10 36 7 202/3,800
2 11 37 7 202/3,800
2 12 35 8 45/1,750~2,750
2 13 36 8 45/1,750~2,750
2 14 37 8 45/1,750~2,750
2 15 35 9 -
2 16 36 9 -
2 17 37 9 -
2 18 35 10 -
2 19 36 10 -
2 20 37 10 -
2 21 35 11 -
2 22 36 11 -
2 23 37 11 -
2 24 35 12 -
2 25 36 12 -
2 26 37 12 -
2 27 35 13 -
2 28 36 13 -
2 29 37 13 -
2 30 35 14 -
2 31 36 14 -
2 32 37 14 -
2 33 35 15 -
2 34 36 15 -
2 35 37 15 -
2 36 35 16 -
2 37 

4 26 75 4 D4CB
4 27 49 5 디젤
4 28 50 5 디젤
4 29 51 5 디젤
4 30 52 5 디젤
4 31 53 5 디젤
4 32 54 5 디젤
4 33 55 5 디젤
4 34 56 5 디젤
4 35 57 5 디젤
4 36 58 5 디젤
4 37 59 5 디젤
4 38 60 5 디젤
4 39 61 5 디젤
4 40 62 5 디젤
4 41 63 5 디젤
4 42 64 5 디젤
4 43 65 5 디젤
4 44 66 5 디젤
4 45 67 5 디젤
4 46 68 5 디젤
4 47 69 5 디젤
4 48 70 5 디젤
4 49 71 5 디젤
4 50 72 5 디젤
4 51 73 5 디젤
4 52 74 5 디젤
4 53 75 5 디젤
4 54 49 6 2,497
4 55 50 6 2,497
4 56 51 6 2,497
4 57 52 6 2,497
4 58 53 6 2,497
4 59 54 6 2,497
4 60 55 6 2,497
4 61 56 6 2,497
4 62 57 6 2,497
4 63 58 6 2,497
4 64 59 6 2,497
4 65 60 6 2,497
4 66 61 6 2,497
4 67 62 6 2,497
4 68 63 6 2,497
4 69 64 6 2,497
4 70 65 6 2,497
4 71 66 6 2,497
4 72 67 6 2,497
4 73 68 6 2,497
4 74 69 6 2,497
4 75 70 6 2,497
4 76 71 6 2,497
4 77 72 6 2,497
4 78 73 6 2,497
4 79 74 6 2,497
4 80 75 6 2,497
4 81 49 7 133/3,600
4 82 50 7 133/3,600
4 83 51 7 133/3,600
4 84 52 7 133/3,600
4 85 53 7 133/3,600
4 86 54 7 133/3,600
4 87 55 7 133/3,600
4 88 56 7 133/3,600
4 89 57 7 133/3,600
4 90 58 7 133/3,600
4 

4 826 65 34 220
4 827 66 34 202
4 828 67 34 207
4 829 68 34 227
4 830 69 34 207
4 831 70 34 227
4 832 71 34 207
4 833 72 34 227
4 834 73 34 207
4 835 74 34 227
4 836 75 34 207
4 837 49 35 4
4 838 50 35 5
4 839 51 35 4
4 840 52 35 5
4 841 53 35 4
4 842 54 35 5
4 843 55 35 4
4 844 56 35 5
4 845 57 35 4
4 846 58 35 4
4 847 59 35 5
4 848 60 35 4
4 849 61 35 5
4 850 62 35 4
4 851 63 35 5
4 852 64 35 4
4 853 65 35 5
4 854 66 35 4
4 855 67 35 4
4 856 68 35 5
4 857 69 35 4
4 858 70 35 5
4 859 71 35 4
4 860 72 35 5
4 861 73 35 4
4 862 74 35 5
4 863 75 35 4
4 864 49 36 -
4 865 50 36 -
4 866 51 36 -
4 867 52 36 -
4 868 53 36 -
4 869 54 36 -
4 870 55 36 -
4 871 56 36 -
4 872 57 36 -
4 873 58 36 -
4 874 59 36 -
4 875 60 36 -
4 876 61 36 -
4 877 62 36 -
4 878 63 36 -
4 879 64 36 -
4 880 65 36 -
4 881 66 36 -
4 882 67 36 -
4 883 68 36 -
4 884 69 36 -
4 885 70 36 -
4 886 71 36 -
4 887 72 36 -
4 888 73 36 -
4 889 74 36 -
4 890 75 36 -
4 891 49 37 -
4 892 50 37 -
4 893 51 37 -
4 894 52 37 -
4 895 53 37 

...

232 142 742 51 -
232 143 743 51 -
232 144 741 52 -
232 145 742 52 -
232 146 743 52 -
232 147 741 53 -
232 148 742 53 -
232 149 743 53 -
232 150 741 54 -
232 151 742 54 -
232 152 743 54 -
232 153 741 55 -
232 154 742 55 -
232 155 743 55 -
232 156 741 56 -
232 157 742 56 -
232 158 743 56 -
233 0 744 4 2.5 VGT
233 1 745 4 2.5 VGT
233 2 746 4 2.5 VGT
233 3 747 4 2.5 VGT
233 4 748 4 2.5 VGT
233 5 749 4 2.5 VGT
233 6 744 5 디젤
233 7 745 5 디젤
233 8 746 5 디젤
233 9 747 5 디젤
233 10 748 5 디젤
233 11 749 5 디젤
233 12 744 6 2,497
233 13 745 6 2,497
233 14 746 6 2,497
233 15 747 6 2,497
233 16 748 6 2,497
233 17 749 6 2,497
233 18 744 7 175/3,600
233 19 745 7 175/3,600
233 20 746 7 175/3,600
233 21 747 7 175/3,600
233 22 748 7 175/3,600
233 23 749 7 175/3,600
233 24 744 8 46/2,000~2,250
233 25 745 8 46/2,000~2,250
233 26 746 8 46/2,000~2,250
233 27 747 8 46/2,000~2,250
233 28 748 8 46/2,000~2,250
233 29 749 8 46/2,000~2,250
233 30 744 9 -
233 31 745 9 -
233 32 746 9 -
233 33 747 9 -
233 34 748 9 -
233 

235 147 758 33 11.4
235 148 759 33 11.4
235 149 760 33 11.4
235 150 756 34 176
235 151 757 34 194
235 152 758 34 194
235 153 759 34 194
235 154 760 34 194
235 155 756 35 4
235 156 757 35 4
235 157 758 35 4
235 158 759 35 4
235 159 760 35 4
235 160 756 36 -
235 161 757 36 -
235 162 758 36 -
235 163 759 36 -
235 164 760 36 -
235 165 756 37 -
235 166 757 37 -
235 167 758 37 -
235 168 759 37 -
235 169 760 37 -
235 170 756 38 -
235 171 757 38 -
235 172 758 38 -
235 173 759 38 -
235 174 760 38 -
235 175 756 39 -
235 176 757 39 -
235 177 758 39 -
235 178 759 39 -
235 179 760 39 -
235 180 756 40 5,095
235 181 757 40 5,095
235 182 758 40 5,095
235 183 759 40 5,095
235 184 760 40 5,095
235 185 756 41 1,950
235 186 757 41 1,950
235 187 758 41 1,950
235 188 759 41 1,950
235 189 760 41 1,950
235 190 756 42 1,840
235 191 757 42 1,840
235 192 758 42 1,840
235 193 759 42 1,840
235 194 760 42 1,840
235 195 756 43 3,100
235 196 757 43 3,100
235 197 758 43 3,100
235 198 759 43 3,100
235 199 760 43 3,100


239 46 777 10 -
239 47 778 10 -
239 48 779 10 -
239 49 773 11 -
239 50 774 11 -
239 51 775 11 -
239 52 776 11 -
239 53 777 11 -
239 54 778 11 -
239 55 779 11 -
239 56 773 12 -
239 57 774 12 -
239 58 775 12 -
239 59 776 12 -
239 60 777 12 -
239 61 778 12 -
239 62 779 12 -
239 63 773 13 -
239 64 774 13 -
239 65 775 13 -
239 66 776 13 -
239 67 777 13 -
239 68 778 13 -
239 69 779 13 -
239 70 773 14 -
239 71 774 14 -
239 72 775 14 -
239 73 776 14 -
239 74 777 14 -
239 75 778 14 -
239 76 779 14 -
239 77 773 15 -
239 78 774 15 -
239 79 775 15 -
239 80 776 15 -
239 81 777 15 -
239 82 778 15 -
239 83 779 15 -
239 84 773 16 -
239 85 774 16 -
239 86 775 16 -
239 87 776 16 -
239 88 777 16 -
239 89 778 16 -
239 90 779 16 -
239 91 773 17 -
239 92 774 17 -
239 93 775 17 -
239 94 776 17 -
239 95 777 17 -
239 96 778 17 -
239 97 779 17 -
239 98 773 18 -
239 99 774 18 -
239 100 775 18 -
239 101 776 18 -
239 102 777 18 -
239 103 778 18 -
239 104 779 18 -
239 105 773 19 -
239 106 774 19 -
239 107 775 19 -


243 156 789 21 FF
243 157 790 21 FF
243 158 791 21 FF
243 159 792 21 FF
243 160 793 21 FF
243 161 794 21 FF
243 162 786 22 자동 8단
243 163 787 22 자동 8단
243 164 788 22 자동 8단
243 165 789 22 자동 8단
243 166 790 22 자동 8단
243 167 791 22 자동 8단
243 168 792 22 자동 8단
243 169 793 22 자동 8단
243 170 794 22 자동 8단
243 171 786 23 맥퍼슨 스트럿
243 172 787 23 맥퍼슨 스트럿
243 173 788 23 맥퍼슨 스트럿
243 174 789 23 맥퍼슨 스트럿
243 175 790 23 맥퍼슨 스트럿
243 176 791 23 맥퍼슨 스트럿
243 177 792 23 맥퍼슨 스트럿
243 178 793 23 맥퍼슨 스트럿
243 179 794 23 맥퍼슨 스트럿
243 180 786 24 멀티링크
243 181 787 24 멀티링크
243 182 788 24 멀티링크
243 183 789 24 멀티링크
243 184 790 24 멀티링크
243 185 791 24 멀티링크
243 186 792 24 멀티링크
243 187 793 24 멀티링크
243 188 794 24 멀티링크
243 189 786 25 디스크
243 190 787 25 디스크
243 191 788 25 디스크
243 192 789 25 디스크
243 193 790 25 디스크
243 194 791 25 디스크
243 195 792 25 디스크
243 196 793 25 디스크
243 197 794 25 디스크
243 198 786 26 디스크
243 199 787 26 디스크
243 200 788 26 디스크
243 201 789 26 디스크
243 202 790 26 디스크
243 203 791 26 디스크
243 204 792 26 디스크
243 205 793 

247 55 804 31 6.1
247 56 803 32 5.1
247 57 804 32 5.1
247 58 803 33 7.9
247 59 804 33 7.9
247 60 803 34 278
247 61 804 34 278
247 62 803 35 5
247 63 804 35 5
247 64 803 36 -
247 65 804 36 -
247 66 803 37 -
247 67 804 37 -
247 68 803 38 -
247 69 804 38 -
247 70 803 39 -
247 71 804 39 -
247 72 803 40 4,910
247 73 804 40 4,920
247 74 803 41 1,915
247 75 804 41 1,915
247 76 803 42 1,380
247 77 804 42 1,380
247 78 803 43 2,940
247 79 804 43 2,940
247 80 803 44 -
247 81 804 44 -
247 82 803 45 -
247 83 804 45 -
247 84 803 46 -
247 85 804 46 -
247 86 803 47 -
247 87 804 47 -
247 88 803 48 4
247 89 804 48 4
247 90 803 49 1,995
247 91 804 49 1,995
247 92 803 50 75
247 93 804 50 75
247 94 803 51 -
247 95 804 51 -
247 96 803 52 -
247 97 804 52 -
247 98 803 53 -
247 99 804 53 -
247 100 803 54 -
247 101 804 54 -
247 102 803 55 5.0
247 103 804 55 4.9
247 104 803 56 288
247 105 804 56 291
248 0 805 4 V8 자연흡기
248 1 806 4 V8 자연흡기
248 2 805 5 가솔린
248 3 806 5 가솔린
248 4 805 6 4,691
248 5 806 6 4,691
248 6 

In [13]:
wb.save("car_spec_output.xlsx")

이제 실패한 모델의 표를 다시 긁어와야 함

In [15]:
# 크롬 웹드라이버 경로 지정
driver = webdriver.Chrome('chromedriver.exe')

#http://auto.danawa.com/auto/modelPopup.php?Type=spec&Lineup=42742&Trims=
# url1과 url2 사이에 모델 number가 들어가야함
url1='http://auto.danawa.com/auto/?Work=model&Model=' 
url2='&Tab=spec'
#table=pd.DataFrame()
tabletitle=[]
tablecol=[[]]
tablerow=[[]]
dtl_model=[[]]
fail_model=[[],[]]
model_kr_name=[[]]
price=[[]]

# 모델 수
cnt=0
#print(model_dic.keys())

for mnum in fail_model[0]:
    try:
        driver.get(url1+mnum+url2)
    except:
        print("조회 페이지 없음")
        continue
    
    html = driver.page_source 
    table_src = BeautifulSoup(html, 'lxml') 
    # 표 부분 가져오기
    
    if table_src.find('div', id='printTable') == None:
        print("테이블 가져오기 실패 => ", url1+mnum+url2)
        fail_model[1].append(mnum)
        continue
    else:
        #table_src = table_src.find('div', id='printTable')
        table_src = table_src.find('div', {'class':'inner'})
        
    # 표에서 세부 모델명 가져오기
    j=0
    try:
        table_src.find('ul', {'class':'modelname'}).find_all("li")
    except:
        print("테이블 가져오기 실패2 => ", url1+mnum+url2)
        fail_model[1].append(mnum)
        continue
    
    print(url1+mnum+url2)
    for dm1 in table_src.find('ul', {'class':'modelname'}).find_all("li"):
        j+=1
        if dm1.text == '-':
            continue
        else:
            dtl_model[cnt].append(dm1.text)
            model_kr_name[cnt].append(model_dic[mnum])
    
    max_i = len(dtl_model[cnt])
    
    # 표에서 가격 가져오기
    i=0
    for pri in table_src.find('ul', {'class':'price'}).find_all("li"):
        if i < max_i:
            price[cnt].append(pri.text)
        else:
            break
        i+=1
        
    
    # 표 그룹 (엔진, 전기, 구동, 연비, 연비(전기), 제원, 적재함, 성능) 가져오기
    group = table_src.find_all('div', {"class":"row group"})
    #print(group)
    
    for grp1 in group:
        tablecol[cnt].append(grp1.span.text)
        i=0
        for grp2 in grp1.find_all("li"):
            if i%j == 0 and len(tabletitle) < 53 : 
                tabletitle.append(grp2.text.replace("\t","").replace("\n",""))
            elif i%j != 0 and i%j < max_i:
                tablerow[cnt].append(grp2.text.replace("\t","").replace("\n",""))
            i+=1
    cnt+=1

    if len(tabletitle) != 53:
        print("테이블 행 개수 예외발생 => url1+mnum+url2")
        fail_model[1].append(mnum)
        break

    if cnt < len(model_dic.keys()):
        dtl_model.append([])
        tablerow.append([])
        tablecol.append([])
        model_kr_name.append([])
        price.append([])
    


In [14]:
for fmno in fail_model[0]:
    print(fmno,model_dic[fmno])


3153 쏘렌토
3239 더 뉴 모닝
3151 쏘나타 (2017)
2129 그랜저HG
2588 K3
3119 제네시스(DH)
2189 코란도 스포츠
2682 K7
3105 뉴 코란도 C
2351 5 Series
3242 뉴 크루즈
2103 쏘나타(YF)
3138 그랜저HG HEV
3080 QM3
3054 트랙스
2180 말리부
3193 LF쏘나타 HEV
2591 ES
3210 SM7
3146 Q50
2214 렉스턴 W
3237 더 뉴 프라이드
2408 A4
2576 Discovery
3231 CR-V
3078 카렌스
3228 CLS
2357 7 Series
3241 더 뉴 i30
3145 A3
2445 Countryman
3147 CLA-Class
2554 XC60
3121 4 Series
3135 2008
3116 쏘울
2645 Altima
3218 Camry
3085 MKZ
2548 V40
2411 A5
3173 QM5
3268 New Q3
3094 A-Class
3197 GLA-Class
3196 Qashqai
3131 K7 하이브리드
2143 아베오
2488 Explorer
2636 Panamera
2545 S80
3168 200
2354 6 Series
2688 Odyssey
2491 MKS
3283 Kuga
2431 TT
2075 에쿠스
2531 3008
3286 B-class
3227 C4 Picasso
3160 QX60
2556 XC70
3189 CT
2633 911
2123 알페온
3310 S60 Cross Country
3171 Cherokee
2155 캡티바
2628 Boxster
3284 LF쏘나타 PHEV
2705 New Continental
3108 K3 쿱
2186 제네시스 쿠페
3139 Pathfinder
2400 SLK-Class
2243 Prius
2847 SL-Class
2474 Focus
2514 DS3
3058 DS5
2508 SRX
3081 Cayman
2149 카마로
2234 RX
3248 Legend
3217 New 